In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

# Verificar si existe el archivo de métricas
metrics_file = "training_metrics.json"
if not os.path.exists(metrics_file):
    print("⚠️ No se encontró el archivo de métricas. Ejecuta primero el entrenamiento:")
    print("python ../train_q_learning.py")
else:
    print("📊 Cargando métricas de entrenamiento...")
    
    with open(metrics_file, "r") as f:
        data = json.load(f)
    
    # Extraer datos
    episodes = [d["episode"] for d in data]
    rewards = [d["reward"] for d in data]
    steps = [d["steps"] for d in data] 
    wins = [d["wins"] for d in data]
    losses = [d["losses"] for d in data]
    draws = [d["draws"] for d in data]
    
    # Calcular métricas adicionales
    total_games = np.array(wins) + np.array(losses) + np.array(draws)
    win_rates = np.array(wins) / total_games * 100
    
    print(f"✅ Datos cargados: {len(data)} puntos de métricas")
    
    # Crear visualizaciones
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Progreso de victorias/derrotas/empates
    axes[0, 0].plot(episodes, wins, label="Victorias", color='green', linewidth=2)
    axes[0, 0].plot(episodes, losses, label="Derrotas", color='red', linewidth=2)
    axes[0, 0].plot(episodes, draws, label="Empates", color='orange', linewidth=2)
    axes[0, 0].set_xlabel("Episodio")
    axes[0, 0].set_ylabel("Partidas Acumuladas")
    axes[0, 0].set_title("📈 Progreso de Entrenamiento")
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Tasa de victoria
    axes[0, 1].plot(episodes, win_rates, label="Tasa de Victoria (%)", color='blue', linewidth=2)
    axes[0, 1].set_xlabel("Episodio")
    axes[0, 1].set_ylabel("Porcentaje de Victorias")
    axes[0, 1].set_title("🎯 Tasa de Victoria")
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([0, 100])
    
    # 3. Recompensas promedio
    # Calcular media móvil para suavizar
    window = min(50, len(rewards)//10)
    if window > 1:
        rewards_smooth = np.convolve(rewards, np.ones(window)/window, mode='valid')
        episodes_smooth = episodes[window-1:]
        axes[1, 0].plot(episodes_smooth, rewards_smooth, label="Recompensa (suavizada)", color='purple', linewidth=2)
    else:
        axes[1, 0].plot(episodes, rewards, label="Recompensa", color='purple', linewidth=2)
    
    axes[1, 0].set_xlabel("Episodio")
    axes[1, 0].set_ylabel("Recompensa")
    axes[1, 0].set_title("💰 Evolución de Recompensas")
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Distribución de duración de partidas
    axes[1, 1].hist(steps, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1, 1].set_xlabel("Pasos por Partida")
    axes[1, 1].set_ylabel("Frecuencia")
    axes[1, 1].set_title("⏱️ Distribución de Duración de Partidas")
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle("🤖 Análisis de Entrenamiento Q-Learning - Connect 4", fontsize=16, y=1.02)
    plt.show()
    
    # Estadísticas finales
    print("\n" + "="*50)
    print("📊 ESTADÍSTICAS FINALES")
    print("="*50)
    print(f"🏆 Victorias: {wins[-1]} ({win_rates[-1]:.1f}%)")
    print(f"💀 Derrotas: {losses[-1]} ({losses[-1]/total_games[-1]*100:.1f}%)")
    print(f"🤝 Empates: {draws[-1]} ({draws[-1]/total_games[-1]*100:.1f}%)")
    print(f"⏱️  Duración promedio: {np.mean(steps):.1f} pasos por partida")
    print(f"🎲 Duración máxima: {max(steps)} pasos")
    print(f"🎲 Duración mínima: {min(steps)} pasos")